In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from transformers.trainer import TrainingArguments
from peft import LoraConfig, AutoPeftModelForCausalLM, get_peft_model, prepare_model_for_int8_training, PeftModel
from trl import SFTTrainer, setup_chat_format
from datasets import load_dataset
from transformers import pipeline

In [ ]:
dataset_name = "ruslanmv/ai-medical-chatbot"
dataset = load_dataset(dataset_name, split="all")
dataset = dataset.shuffle(seed=42).select(range(1000))

In [ ]:
with open('hf_token.key', 'r') as f:
    hf_token = f.read()

base_model = "meta-llama/Meta-Llama-3-8B-Instruct"
new_model = "llama-3-8b-counsel"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model, padding='max_length', truncation=True, token = hf_token)
# Adding a special token for pad token so that eos token can be recognized 
# (https://github.com/unslothai/unsloth/issues/416)
# https://github.com/huggingface/transformers/issues/22794
# https://github.com/huggingface/transformers/issues/23230
tokenizer.add_special_tokens({"pad_token": "<|reserved_special_token_0|>"})
tokenizer.padding_side = "right"
tokenizer.model_max_length = 256

In [ ]:
def format_chat_template(row):
    row_json = [
        {"role" : "user", "content": row['Patient']},
        {"role" : "assistant", "content": row['Doctor']}
    ]

    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

In [ ]:
dataset = dataset.map(
            format_chat_template,
            num_proc=4
        )

dataset = dataset.train_test_split(test_size=0.1)

In [ ]:
# # QLoRA Config for 4-bit quntization
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16,
#     bnb_4bit_use_double_quant=True
# )

# For 8 bit quantization
bnb_config = BitsAndBytesConfig(load_in_8bit=True, llm_int8_threshold=200.0)

In [ ]:
# Load Model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config = bnb_config,
    torch_dtype = torch.bfloat16,
    device_map={'':torch.cuda.current_device()}
)

model.config.use_cache=False
model.config.pad_token_id = tokenizer.pad_token_id # Updating the model config to use the special pad token

In [ ]:
peft_config = LoraConfig(
    r=64,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type = "CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj'],
    # target_modules=['k_proj', 'q_proj', 'v_proj', 'o_proj'],
)

model = get_peft_model(model, peft_config)

In [ ]:
training_arguments = TrainingArguments(
    output_dir=new_model,
    overwrite_output_dir=True,
    bf16=True,
    do_eval=True,
    evaluation_strategy="steps",
    eval_steps=0.1,
    learning_rate=2e-4,
    logging_steps=5,
    logging_strategy="steps",
    log_level="info",
    save_strategy="epoch",
    num_train_epochs=2,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    warmup_steps=10,
    group_by_length=True,
    report_to="none",
    seed=42
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    tokenizer=tokenizer,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    dataset_text_field="text",
    peft_config=peft_config,
    max_seq_length=tokenizer.model_max_length,
    packing= False
)

In [ ]:
trainer.train()

### Saving the trained model

In [ ]:
trainer.model.save_pretrained(new_model)

### Merging the base model with the adapter to get full model

In [ ]:
base_model = "meta-llama/Meta-Llama-3-8B-Instruct"
new_model = "llama-3-8b-counsel"

In [ ]:
# Load Model
base_model_reload = AutoModelForCausalLM.from_pretrained(
    base_model,
    return_dict=True,
    low_cpu_mem_usage=True,    
    trust_remote_code=True,
    torch_dtype = torch.bfloat16,
    device_map={"":torch.cuda.current_device()}
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model, padding='max_length', truncation=True, token = hf_token)
tokenizer.add_special_tokens({"pad_token": "<|reserved_special_token_0|>"})
tokenizer.padding_side = "right"
tokenizer.model_max_length = 256

Merge adapter with the base model

In [ ]:
model = PeftModel.from_pretrained(base_model_reload, new_model)

In [ ]:
model = model.merge_and_unload()
model.config.pad_token_id = tokenizer.pad_token_id # Updating the model config to use the special pad token

In [ ]:
model.save_pretrained("llama-3-8b-chat-doctor")
tokenizer.save_pretrained("llama-3-8b-chat-doctor")

### Load merged Model and Tokenizer for Inference

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    "llama-3-8b-chat-doctor",
    torch_dtype = torch.bfloat16,
    device_map={'':torch.cuda.current_device()}
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:06<00:00,  1.66s/it]


In [4]:
tokenizer = AutoTokenizer.from_pretrained("llama-3-8b-chat-doctor")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# 128001  =<|end_of_text|> (terminating cahracter of llama-3-8b-base)
# 128009 = <|eot_id| >(terminating cahracter of llama-3-8b-instruct)

In [7]:
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>"),
]

In [12]:
model.config.use_cache = True

messages = [
    {
        "role": "user",
        "content": "Hello doctor, I have acne. How can I get rid of these?"
    }
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors='pt').to(model.device)

generation_config = model.generation_config
generation_config.pad_token_id = tokenizer.pad_token_id
generation_config.repetition_penalty = 1.5

outputs = model.generate(
    **inputs,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
    num_return_sequences=1,
    generation_config=generation_config
)

text = tokenizer.decode(outputs[0], skip_special_tokens=False)

print(text.split("assistant")[1])

<|end_header_id|>

Hi,I read your query with concern and understand that you are suffering from Acne.I would advise to go for a combination treatment comprising of topical antibiotics like Cetraben Gel or Doxy 100mg once daily alongwith oral anti-acnes (Antibiotics) in the form Tab doxycycline twice day.If this does not work then we may need more powerful medicines which will be advised after consultation.Please consult me back if there is any further clarification required.Regards - Dr Prashant Bhosle M.D.,Dermatologist & Sex Specialist Online Doctor Consultation @ http://www.drprshs.com/ Contact Number: +91-9890XXXXX Email ID : [dr.pr.sh@sifymail.com](mailto:drsps@gmail.in), drpsskolkata@ymail.com Website-http://www.drpsss.kol.co.nf Please note my contact details before asking question as it helps us both.Hopefully answer was helpful.Take care RegardsDr.Prashanth BhosaleMDSpecialist Dermatology &SexologyOnline Clinic Available on www.doctoralia.org<|eot_id|>
